In [0]:
# Databricks Notebook: Gold -> Signals (이동평균/크로스 신호 계산)
# 목적: Gold 집계(avg_price)를 기반으로 심볼별 MA(50/200)와 크로스/추세 신호를 계산해 Signals 테이블로 업서트

from pyspark.sql import functions as F
from pyspark.sql.window import Window
from delta.tables import DeltaTable

# =========================
# (A) 환경/대상 테이블 설정
# =========================
CATALOG = "demo_catalog"
SCHEMA  = "demo_schema"
GOLD    = f"{CATALOG}.{SCHEMA}.gold_signals"     # 5분 버킷 등 윈도우 집계 결과
SIGNALS = f"{CATALOG}.{SCHEMA}.signals_charts"   # 신호 결과 저장 테이블

# =========================
# (B) 타깃 테이블 생성(없으면)
#  - 스키마 명시: 이동평균/신호/상태 컬럼 포함
# =========================
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {SIGNALS} (
  bucket_start TIMESTAMP,  -- 버킷 시작 시각
  bucket_end   TIMESTAMP,  -- 버킷 종료 시각
  symbol       STRING,     -- 자산 심볼
  avg_price    DOUBLE,     -- 버킷 평균 종가
  ma_50        DOUBLE,     -- 이동평균(최근 50 버킷)
  ma_200       DOUBLE,     -- 이동평균(최근 200 버킷)
  cross_signal STRING,     -- Golden Cross / Dead Cross / Neutral
  above_ma200  BOOLEAN     -- 현재 avg_price가 MA200 위인지 여부
) USING DELTA
""")

gold = spark.table(GOLD)

# =========================
# (C) 윈도우 정의
#  - 심볼별 시간 순서로 이동창 구성(rowsBetween)
# =========================
w50  = Window.partitionBy("symbol").orderBy("bucket_start").rowsBetween(-49, 0)
w200 = Window.partitionBy("symbol").orderBy("bucket_start").rowsBetween(-199, 0)

# =========================
# (D) 신호 계산 DataFrame
#  - latest_signal 컬럼은 사용/전파하지 않음
#  - (symbol, bucket_start) 단위로 중복 방지
# =========================
signals_df = (
  gold
    .select("bucket_start", "bucket_end", "symbol", "avg_price")  # latest_signal 제외
    .withColumn("ma_50",  F.avg("avg_price").over(w50))
    .withColumn("ma_200", F.avg("avg_price").over(w200))
    .withColumn(
        "cross_signal",
        F.when(F.col("ma_50") > F.col("ma_200"), F.lit("Golden Cross"))
         .when(F.col("ma_50") < F.col("ma_200"), F.lit("Dead Cross"))
         .otherwise(F.lit("Neutral"))
    )
    .withColumn("above_ma200", F.col("avg_price") > F.col("ma_200"))
    .dropDuplicates(["symbol", "bucket_start"])
)

# =========================
# (E) Delta MERGE (idempotent upsert)
#  - 조인 키: (symbol, bucket_start)
#  - 존재 시 UPDATE, 없으면 INSERT
# =========================
target = DeltaTable.forName(spark, SIGNALS)

(
  target.alias("t")
    .merge(
      signals_df.alias("s"),
      "t.symbol = s.symbol AND t.bucket_start = s.bucket_start"
    )
    .whenMatchedUpdateAll()
    .whenNotMatchedInsertAll()
    .execute()
)

print(f"Signals upserted into {SIGNALS}")